In [23]:
import pandas as pd
import numpy as np
import xlwings as xw
import glob

In [50]:
xl = r'C:\Users\emilh\Desktop\Data_split\Full data\AEPrec_sABRe_data_TPsABR21_BILATERAL_2021-Aug-18_13.09.55.xlsx'

try:
    excel_app=xw.App(visible=False)
    wb=excel_app.books.open(xl)
    for sheet in wb.sheets:
        sheet.api.Copy()
        wb_new=xw.books.active
        wb_new.save(f'{xl+sheet.name}.xlsx')
        wb_new.close()
finally:
    excel_app.quit()

In [59]:
path=r'C:\Users\emilh\Desktop\Data_split'
allfiles=glob.glob(path+"/*.xlsx")
dataframes = []

for files in allfiles:
    df=pd.read_excel(files)
    df.drop(['Time (msec)', 'cond_wa (nV)','rare_wa (nV)', 'contra_wa (nV)'], inplace=True, axis=1)
    #df.insert(0, "Signal", "1")
    for col in df.columns:
        new_df = pd.DataFrame(df[col])
        dataframes.append(new_df)
        new_df.to_excel('data.xlsx')

TypeError: can only concatenate str (not "int") to str